# Coordle Idea
1. Get query, e.g. "What is love?"
2. Tokenize query
3. Create a vector using word2vec of the tokens by summing them, or finding the average or whatever.
   lets call it the query vector. 
4. Find sentence vectors that are the closesest to the query vector.
5. Return the papers corresponding to said sentence vectors.

# Coordle Frontend
💩

# Coordle Backend

In [23]:
import pandas as pd 
import numpy as np
from importlib import reload
import nltk
import spacy
import en_core_sci_lg # Biomedical word embeddings
from utils import clean_text, load_pickle, save_pickle
from nltk.corpus import stopwords as _stopwords
from tqdm import tqdm
from collections import deque
from copy import deepcopy
from collections.abc import Iterable
from typing import Union
from pprint import pprint
from itertools import chain

import os
import json
import re 
from string import punctuation as PUNCTUATION
from nltk.corpus import stopwords as _stopwords
from gensim.models import Word2Vec
from os.path import join as join_path
from gensim.models.callbacks import CallbackAny2Vec

In [3]:
df = pd.read_csv('data/cord-19-data.csv')

In [4]:
class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

    def __init__(self, output_dir: str, prefix: str, logs_filename: str):
        self.output_dir = output_dir
        self.prefix = prefix
        self.logs_filename = logs_filename
        self.epoch = 0

    def on_epoch_end(self, model):
        cum_loss = model.get_latest_training_loss()
        if self.epoch == 0:
            loss = cum_loss
        else:
            loss = cum_loss - self.loss_previous_step
        self.loss_previous_step = loss
        with open(join_path(self.output_dir, self.logs_filename), 'a+') as file:
            file.write(f'Epoch #{self.epoch}, loss: {loss}\n')
        
        output_path = join_path(self.output_dir, f'{self.prefix}_epoch_{self.epoch}.model')
        model.save(output_path)
        self.epoch += 1    

class DocEpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

    def __init__(self, output_dir: str, prefix: str, start_epoch: int = 1):
        self.output_dir = output_dir
        self.prefix = prefix
        self.epoch = start_epoch

    def on_epoch_end(self, model):        
        output_path = join_path(self.output_dir, f'{self.prefix}_epoch_{self.epoch}.model')
        model.save(output_path)
        self.epoch += 1    

In [5]:
# Load the last trained model
model = Word2Vec.load(join_path('models-word2vec', 'w2v_model_epoch_29.model'))
word_to_int = {word:i for i, word in enumerate(model.wv.index2word)}
int_to_word = np.array(model.wv.index2word)

In [133]:
# %%time
import coordle_backend
reload(coordle_backend)
from coordle_backend import SentVectorDoc, Index, RecursiveDescentParser

def test_SentVectorDoc():
    docsample = df[df.cord_uid == 'vs5yondw']
#     doc = SentVectorDoc(docsample['cord_uid'], docsample['title'])
#     doc, _ = doc.fit(docsample['body_text'])
    gaga = Index()
    gaga.build_from_df(
        docsample,
#         df[:32],
        'cord_uid',
        'title',
        'body_text', 
        verbose=True, 
        use_multiprocessing=False,
        workers=-1
    )
    
    return gaga
#     return doc

def test_Index():
    coordle = Index()
    coordle.build_from_df(
        df.iloc[:4196],
        'cord_uid',
        'title',
        'body_text', 
        verbose=True, 
        use_multiprocessing=True,
        workers=-1
    )
    return coordle

index = test_Index()
# fuck = test_SentVectorDoc()

Text cleaning initilized on 16 workers


Adding to index: 100%|██████████| 4196/4196 [00:19<00:00, 219.56it/s]


In [127]:
# save_pickle(index, 'coordle/coordle_index_8k_0904.p')

In [128]:
query = 'retarded white woman'
docs1, score = index.search(query, verbose=True)
print(docs1)
print()
# docs2 = index.advanced_search(query, verbose=True)
# print(docs2)

Query tokens:  ['retarded', 'white', 'woman']
[8bnny6hj, zp9k1k3z, 5xswwney, fdtwagr1, 1rzbaj02, hnl4a33n, 15mv5yx7, 2ole6ya7, vs5yondw, j1iylpgm, eyk015n3, bgygebgx, mnhg329v, y2ruobm0, uq88w4fm, i94lyfsh, lno01041, y2uhnlpd, 9jpc42h1, ensep6lk, qf32xs4m, 34jn37sb, p8bzkmas, y3hgnb6u, rwlryr0h, 67kyfybe, vdk2pxkk, 0b1qaecu, vo2oe2wr, uhrijlyz, 9hru9fyg, i6vuhaiv, mj9ea464, ret6iq3o, piwdsazx, fu0x6bqb, qg2pb884, pq0atbfg, x4fbph3d, ootj52fs, 8ext7xa7, dksh41wz, mvv1rg68, sb43y1cl, x8uzlsn7, gu0930s3, pbhdzshl, 7lzyvzg2, 5yrouyvo, qwh8ei60, fvrhzc7x, h1gnp62r, 8kooncil, 8rs8ilio, oir3rlb7, 5lmrthmb, wwf90zxt]



In [129]:
query = 'retarded    (white AND woman)'
querytokens = re.split('([^a-zA-Z0-9])', query)
# Gotta do this to capture parenthesis
querytokens = chain.from_iterable([t.split() for t in querytokens])

print(list(querytokens))

['retarded', '(', 'white', 'AND', 'woman', ')']


In [135]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import SentVectorDoc, Index, RecursiveDescentParser

queries = [
    'retarded!!!',
#     '(',
#     ')',
    'retarded (white AND (woman NOT man))',
    'retarded (white AND white) man',
#     'retarded OR white OR woman',
#     'retarded white AND woman',
#     'retarded OR white NOT woman',
#     'retarded (white NOT woman)',
#     'retarded (white NOT woman)',
#     'OR retarded AND white woman',
#     'retarded AND AND white NOT woman',
#     'retarded (white NOT woman) AND',
#     ')retarded ((white NOT woman) AND',
#     'retarded ((white NOT woman)',
#     'AND retarded)) ((white NOT woman) NOT',
]

rdp = RecursiveDescentParser(index.docmap)
for query in queries:
    errmsgs = []
    tokens = rdp.get_logical_querytokens(query)
    pass_ = rdp.assert_query(tokens, errmsgs)
    
#     print(tokens)
    print(parenthesis_handler(tokens))
#     print(errmsgs)
    print(pass_)
    print()
    

deque(['retarded'])
True

deque(['retarded', 'OR', deque(['white', 'AND', deque(['woman', 'NOT', 'man'])])])
True

deque(['retarded', 'OR', deque(['white', 'AND', 'white']), 'OR', 'man'])
True



In [139]:
query = 'retarded (white AND (woman NOT man))'

docs, scores = rdp.search(query)
n = 100
for doc, score in zip(docs[:n], scores[:n]):
    print(f'{doc.uid}   {str(doc.title)[:69]:<69}   {score:.4f}')

alm3p31f   Chapter 3 Immunobiological aspects of vaccines in pregnancy: Maternal   6.0400
94anrxyw   Chapter 47 Infections in Pregnancy                                      5.2435
8p4t1dr6   Emergency Caesarean delivery in a patient with confirmed coronavirus    3.5063
7xef0hdr   96 The Lungs in Obstetric and Gynecologic Diseases                      2.4528
0vt8bxpn   Chapter 45 Gender Differences in Emerging Infectious Diseases           1.9134
2qg9vuvd   Chapter 3 Blood Vital but Potentially Dangerous                         1.8781
zp9k1k3z   Clinical and immunologic studies in identical twins discordant for sy   1.8248
9olk4czm   Association of angiotensin-converting enzyme 2 gene A/G polymorphism    1.6936
1jwsq044   3.07 Alkaloid Glycosidase Inhibitors                                    1.6499
qw47pf9r   VII Digestive System 1                                                  1.5487
bv7udg9k   Chapter 13 Transmission of Infectious Diseases Through Breast Milk an   1.4565
gtcrwqgl  

In [95]:
print(f'Cardinality 1: {len(docs1)}')
print(f'Cardinality 2: {len(docs2)}')

docs1_list = sorted(docs1, key=lambda x: x.uid)
docs2_list = sorted(docs2, key=lambda x: x.uid)

# Set equal gave unexpceted results
print(f'Equal?: {all([d1 == d2 for d1, d2 in zip(docs1_list, docs2_list)])}')

Cardinality 1: 57
Cardinality 2: 3
Equal?: False


In [96]:
with open('textfile.txt', 'w+') as f:
#     uid = docs2_list[0].uid
#     uid = 'dlh93ax6'
#     uid = 'zp9k1k3z'
#     uid = 'vs5yondw'
    uid = 'i94lyfsh'
    
    print(f'dumping {uid}')
    f.write(df[df.cord_uid == uid].body_text.values[0].lower())

dumping i94lyfsh


In [ ]:
docs, scores = index.search('retarded white woman', verbose=True)
# docs, scores = index.search('!', verbose=True)
n = 16
print()
for doc, score in zip(docs[:n], scores[:n]):
    print(f'{doc.uid}   {str(doc.title)[:69]:<69}   {score:.4f}')

In [ ]:
raise ValueError

In [ ]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import SentVectorDoc, Index, AI_Index

def most_similar(token):
    if token in model.wv:
        return model.wv.most_similar(token)
    return []

def test_AI_Index(model):
    coordle = AI_Index(most_similar, 3)
    coordle.build_from_df(
        df.iloc[:1024],
        'cord_uid',
        'title',
        'body_text',
        use_multiprocessing=True,
        workers=-1,
        verbose=True
    )
    return coordle

ai_index = test_AI_Index(model)
            

In [ ]:
docs, scores = ai_index.search('retarded white woman', verbose=True)
n = 16
print()
for doc, score in zip(docs[:n], scores[:n]):
    print(f'{doc.uid}   {doc.title[:80]:<80}   {score:.4f}')

## Multiprocessing showdown

In [ ]:
from multiprocessing import Pool
from time import time

In [ ]:
t0 = time()
with Pool(None) as p:
    cleaned_texts = list(tqdm(p.imap(clean_text, df.body_text), position=0, total=len(df)))
print(f'Text cleaning with multiprocessing took {time()-t0:.2f} seconds')

In [ ]:
t0 = time()
gen = tqdm((clean_text(text) for text in df.body_text), position=0, total=len(df))
cleaned_texts = list(gen)
print(f'Naïve text cleaning took {time()-t0:.2f} seconds')

In [ ]:
t0 = time()
with Pool(None) as p:
    cleaned_texts = list(tqdm(p.imap(clean_text, df.body_text[:2048]), position=0, total=2048))
print(f'Text cleaning with multiprocessing took {time()-t0:.2f} seconds')

In [ ]:
t0 = time()
gen = tqdm((clean_text(text) for text in df.body_text[:2048]), position=0, total=2048)
cleaned_texts = list(gen)
print(f'Naïve text cleaning took {time()-t0:.2f} seconds')